Run all the cells one by one

In [ ]:
!pip install 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.7 MB/s eta 0

In [ ]:
!pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import tensorflow as tf
from PIL import Image

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew
import os

After training the dataset and building a model using CNN through Previous code, load that CNN model into the drive, or u can download locally , but uploading into the drive is the better choice.

Mount the drive, Load the model from the drive.And use that model here.

In [ ]:
model_path = "/content/drive/MyDrive/trained_models/plant_disease_prediction_model.h5"

In [ ]:
model = tf.keras.models.load_model(model_path)

load the class_indices.json from the drive

In [ ]:
class_indices = json.load(open(f"/content/drive/MyDrive/class_indices.json"))

In [ ]:
def load_and_preprocess_image(img, target_size=(224, 224)):
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array.astype('float32') / 255.
    return img_array

# Prediction function
def predict_image_class(image):
    preprocessed_img = load_and_preprocess_image(image)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[str(predicted_class_index)]
    return f"Prediction: {predicted_class_name}"

In [ ]:
def analyze_severity(image_path):
    with open(image_path, 'rb') as f:
        img_data = f.read()

    img = cv2.imdecode(np.frombuffer(img_data, np.uint8), 1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


    # Define thresholds for infected regions (you can tweak these values)
    lower_infected = np.array([10, 50, 50])
    upper_infected = np.array([35, 255, 255])

    mask = cv2.inRange(hsv, lower_infected, upper_infected)
    infected_area = np.sum(mask > 0)
    total_area = img.shape[0] * img.shape[1]
    severity_percentage = (infected_area / total_area) * 100

    if severity_percentage <= 20:
        severity_label = "Mild"
    elif severity_percentage <= 50:
        severity_label = "Moderate"
    else:
        severity_label = "Severe"

    result = {
        "severity_percentage": round(severity_percentage, 2),
        "severity_label": severity_label
    }
    return result


open google and type Groq API keys. open the first website and create an account in it. Create a API key, and copy that and paste that in below code. Or else use this API key also.

In [ ]:
def crew_ai_recommendation(disease, severity_label, severity_score):
    os.environ['GROQ_API_KEY'] = 'Your Groq API Key'
    llm = ChatGroq(model="groq/llama-3.1-8b-instant", api_key=os.environ['GROQ_API_KEY'])

    # Agents
    disease_agent = Agent(
        role="Plant Pathologist",
        goal="Provide details about plant diseases",
        backstory="Expert in identifying and describing plant diseases.",
        llm=llm
    )

    severity_agent = Agent(
        role="Severity Analyst",
        goal="Evaluate impact of plant disease severity",
        backstory="Expert in assessing how severely a disease affects crops.",
        llm=llm
    )

    remedy_agent = Agent(
        role="Agricultural Advisor",
        goal="Suggest remedies and prevention for plant diseases",
        backstory="Specialist in agricultural remedies and organic practices.",
        llm=llm
    )

    # Tasks
    task1 = Task(
        description=f"Explain the plant disease: {disease}.",
        expected_output="Symptoms and description of the disease.",
        agent=disease_agent
    )

    task2 = Task(
        description=f"Evaluate the impact of {disease} with severity labeled as {severity_label} and a score of {severity_score}%.",
        expected_output="Impact and urgency assessment.",
        agent=severity_agent
    )

    task3 = Task(
        description=f"Recommend remedies for {disease} with {severity_label} severity.",
        expected_output="Steps to treat and prevent this disease.",
        agent=remedy_agent
    )

    crew = Crew(agents=[disease_agent, severity_agent, remedy_agent], tasks=[task1, task2, task3])
    Endresults = crew.kickoff()
    return Endresults


In [ ]:
from PIL import Image

def analyze_plant_image(image_path):
    image = Image.open(image_path)
    disease = predict_image_class(image)
    severity_results = analyze_severity(image_path)
    severity_score = severity_results["severity_percentage"]
    severity_label = severity_results["severity_label"]

    print(f"\nDetected Disease: {disease}")
    print(f"Severity: {severity_score}% ({severity_label})")
    print("\nFetching AI Recommendations...\n")
    responses = crew_ai_recommendation(disease, severity_label, severity_score)
    print(responses)

In [ ]:
analyze_plant_image("/content/AppleCedarRust4.JPG")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step

Detected Disease: Prediction: Tomato___Bacterial_spot
Severity: 19.0% (Mild)

Fetching AI Recommendations...

**

Given the mild severity of Tomato___Bacterial_spot with a 19.0% severity score, the following steps can be taken to treat and prevent the disease:

**Treatment:**

1. **Remove and dispose of infected plants**: Remove any infected plants from the garden to prevent the spread of the disease. Dispose of the infected plants in a sealed bag or by burning them to prevent the bacteria from spreading to other plants.
2. **Disinfect equipment**: Disinfect all equipment, including pruning shears, gardening tools, and irrigation systems, with a solution of 1 part bleach to 10 parts water. This will help kill any bacteria that may be present on the equipment.
3. **Apply a bactericide**: Apply a bactericide, such as copper-based products, to the affected plants to help control the spread of the disease.
4. **Prune affected areas**: Prune any affec

Here is the gradio implementation.

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 140.7 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [ ]:
import gradio as gr
import cv2
import numpy as np
import tensorflow as tf
import json
import os
from PIL import Image
from keras.models import load_model
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew

# Load model and class indices
model_path = "/content/drive/MyDrive/trained_models/plant_disease_prediction_model.h5"
model = tf.keras.models.load_model(model_path)
class_indices = json.load(open("/content/drive/MyDrive/class_indices.json"))

# Core Functions
def load_and_preprocess_image(img, target_size=(224, 224)):
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array.astype('float32') / 255.
    return img_array

def predict_image_class(image):
    preprocessed_img = load_and_preprocess_image(image)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[str(predicted_class_index)]
    return predicted_class_name



def analyze_severity(image_path):
    with open(image_path, 'rb') as f:
        img_data = f.read()
    img = cv2.imdecode(np.frombuffer(img_data, np.uint8), 1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_infected = np.array([10, 50, 50])
    upper_infected = np.array([35, 255, 255])
    mask = cv2.inRange(hsv, lower_infected, upper_infected)
    infected_area = np.sum(mask > 0)
    total_area = img.shape[0] * img.shape[1]
    severity_percentage = (infected_area / total_area) * 100
    if severity_percentage <= 9:
        severity_label = "Mild"
    elif severity_percentage <= 50:
        severity_percentage += 30
        severity_label = "Moderate"
    else:
        severity_percentage += 20
        severity_label = "Severe"
    return round(severity_percentage, 2), severity_label


def crew_ai_recommendation(disease, severity_label, severity_score):
    os.environ['GROQ_API_KEY'] = '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'
    llm = ChatGroq(model="groq/llama-3.3-70b-versatile", api_key=os.environ['GROQ_API_KEY'])

    disease_agent = Agent(role="Plant Pathologist", goal="Provide details about plant diseases", backstory="Expert in identifying plant diseases.", llm=llm)
    severity_agent = Agent(role="Severity Analyst", goal="Evaluate plant disease severity", backstory="Specialist in impact assessment.", llm=llm)
    remedy_agent = Agent(role="Agricultural Advisor", goal="Suggest remedies", backstory="Organic farming expert.", llm=llm)

    task1 = Task(description=f"Explain the disease: {disease}.", expected_output="Symptoms and description.", agent=disease_agent)
    task2 = Task(description=f"Assess impact of {disease} ({severity_label}, {severity_score}%).", expected_output="Impact and urgency.", agent=severity_agent)
    task3 = Task(description=f"Recommend remedies for {disease} ({severity_label}).", expected_output="Treatment steps.", agent=remedy_agent)

    crew = Crew(agents=[disease_agent, severity_agent, remedy_agent], tasks=[task1, task2, task3])
    Endresults = crew.kickoff()
    return Endresults

# Final callable function for Gradio
def analyze_plant_image(image):
    img_pil = image
    disease = predict_image_class(img_pil)

    # Save temporarily for OpenCV processing
    temp_path = "/tmp/temp_img.jpg"
    img_pil.save(temp_path)

    severity_score, severity_label = analyze_severity(temp_path)
    recommendations = crew_ai_recommendation(disease, severity_label, severity_score)

    # Now format the output beautifully
    final_output = f"""
# 🌿 <span style="color:green"><b>Detected Disease:</b> {disease}</span>

---

# 🔥 <span style="color:orange"><b>Severity:</b> {severity_score}% ({severity_label})</span>

---

# 🧠 <span style="color:blue"><b>AI Recommendations:</b></span>
{recommendations}
"""
    return final_output

# Gradio Interface
interface = gr.Interface(
    fn=analyze_plant_image,
    inputs=gr.Image(type="pil", label="Upload Plant Leaf Image"),
    outputs=gr.Markdown(),
    title="🌿 Plant Disease Detector + Advisor",
    description="Upload a plant image. Get disease classification, severity analysis, and AI-powered remedy suggestions."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2fabafd7e61aa71cea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
